In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import BayesianRidge
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import BaggingRegressor

import warnings
warnings.filterwarnings(action='ignore')



In [2]:
# get data

train = pd.read_csv('df - train_set.csv')
test = pd.read_csv('df - test_set.csv')

train = train[(train['Commodities'] == 'APPLE GOLDEN DELICIOUS')]
del train['Commodities'] 

train.head()

,Province,Container,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,CAPE,M4183,1L,18.3,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
7,CAPE,JG110,2M,11.0,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55
24,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11
40,CAPE,M4183,1S,18.3,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51
69,EASTERN CAPE,IA400,1S,400.0,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50


In [3]:
del test['Commodities']
test.head()

,Index,Province,Container,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
0,1,W.CAPE-BERGRIVER ETC,EC120,1M,12.0,2020-07-09,128.0,136.0,5008.0,38,456.0,0
1,2,W.CAPE-BERGRIVER ETC,M4183,1X,18.3,2020-01-20,220.0,220.0,1760.0,8,146.4,2
2,3,W.CAPE-BERGRIVER ETC,EC120,1S,12.0,2020-08-19,120.0,120.0,720.0,6,72.0,45
3,4,W.CAPE-BERGRIVER ETC,M4183,1M,18.3,2020-05-06,160.0,160.0,160.0,1,18.3,8
4,5,W.CAPE-BERGRIVER ETC,M4183,1L,18.3,2020-05-04,140.0,160.0,14140.0,100,1830.0,19


In [4]:
train.shape

(1952, 12)

In [198]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Index           685 non-null    int64  
 1   Province        685 non-null    object 
 2   Container       685 non-null    object 
 3   Size_Grade      685 non-null    object 
 4   Weight_Kg       685 non-null    float64
 5   Commodities     685 non-null    object 
 6   Date            685 non-null    object 
 7   Low_Price       685 non-null    float64
 8   High_Price      685 non-null    float64
 9   Sales_Total     685 non-null    float64
 10  Total_Qty_Sold  685 non-null    int64  
 11  Total_Kg_Sold   685 non-null    float64
 12  Stock_On_Hand   685 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 69.7+ KB


In [5]:
train.describe()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
count,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000
mean,40.460912,174.307377,215.648053,20053.533811,174.510758,2960.176332,408.393955,6.778893
std,99.655169,373.553578,433.546159,39005.069445,308.810797,6097.416527,724.450582,2.248744
min,3.000000,2.000000,5.000000,5.000000,1.000000,3.000000,0.000000,0.250000
25%,9.000000,50.000000,60.000000,1325.000000,12.000000,219.600000,9.000000,5.460000
50%,12.000000,80.000000,108.000000,5495.000000,64.000000,853.500000,126.500000,6.670000
75%,18.300000,127.250000,160.000000,21082.500000,200.000000,3093.525000,468.000000,8.280000
max,400.000000,2300.000000,3300.000000,369464.000000,4237.000000,74000.000000,6400.000000,21.240000


In [7]:
train['Province'].unique()

array(['CAPE', 'W.CAPE-BERGRIVER ETC', 'EASTERN CAPE', 'NATAL',
       'WEST COAST', 'TRANSVAAL', 'ORANGE FREE STATE'], dtype=object)

In [8]:
train['Container'].unique()

array(['M4183', 'JG110', 'JE090', 'IA400', 'EC120', 'AC030', 'M6125',
       'EF120', 'DT063', 'M9125', 'EG140'], dtype=object)

In [275]:
train['Size_Grade'].unique()

array(['1L', '2M', '2S', '1S', '1M', '1X', '2L', '2U', '2X', '1U'],
      dtype=object)

In [5]:
# DUMMY

def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [6]:
# PREPROCESSING 
seed = 1
def preprocess_inputs(df, return_df=False):
    df = df.copy()
    
    # CLEAN PROVINCE COLUMN
    
    df['Province'] = df['Province'].str.replace(' ', '_')
    df['Province'] = df['Province'].str.replace('.', '_')
    df['Province'] = df['Province'].str.replace('-', '_')
    
    # DATE ENCODING
    # Split 'Date' column into year, month and day columns 

    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%d.%m.%Y')
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    df['day'] = pd.DatetimeIndex(df['Date']).day

    df = df.drop(['Date'], axis = 1) 
    
    # BINARY ENCODING
    
    df['year'] = df['year'].replace({2020: 1, 2019: 0})
        
    # ONE-HOT ENCODING
    for column in ['Province', 'Container']:
        df = onehot_encode(df, column)
        
    # ORDINAL ENCODING
    enc = OrdinalEncoder()
    df[['Size_Grade']] = enc.fit_transform(df[['Size_Grade']])
        
    if return_df==True:
        ## for training dataset
        # REORDER COLUMNS SO THAT OUR DEPENDENT VARIABLE IS THE LAST COLUMN OF THE DATAFRAME
        if 'avg_price_per_kg' in df.columns:
            column_titles = [col for col in df.columns if col!= 'avg_price_per_kg'] + ['avg_price_per_kg']
            df = df.reindex(columns = column_titles)
            
        return df
    
        ## for training dataset
        # REORDER COLUMNS SO THAT OUR DEPENDENT VARIABLE IS THE LAST COLUMN OF THE DATAFRAME
    elif 'avg_price_per_kg' in df.columns:
        column_titles = [col for col in df.columns if col!= 'avg_price_per_kg'] + ['avg_price_per_kg']
        df = df.reindex(columns = column_titles)
    
        # SPLIT DATA INTO PREDICTORS AND TARGET
    
        y = df['avg_price_per_kg']
        X = df.drop('avg_price_per_kg', axis=1)
        y = np.array(y)
        

        # SCALE
        #x_scaler = StandardScaler()
        #y_scaler = StandardScaler()

        #X_scaled = x_scaler.fit_transform(X) 
        #y_scaled = y_scaler.fit_transform(y.reshape(-1, 1)) 
        
        # TRAIN TEST SPLIT
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, shuffle=False, random_state=seed)
        
        return X_train, X_test, y_train, y_test
    
        
    else:
        return df
    

In [7]:

X_train, X_test, y_train, y_test = preprocess_inputs(train)

In [8]:
# TRAIN A FEW MODELS

models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(random_state= seed),
    "                         Random Forest": RandomForestRegressor(n_estimators=442, random_state= seed),
    "                     Gradient Boosting": GradientBoostingRegressor(n_estimators=442, random_state= seed),
    "                               XGBoost": XGBRegressor(max_depth=3,min_child_weight=3,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=442, learning_rate=0.1, random_state= seed),
    "                         BayesianRidge": BayesianRidge(),
    "                        HuberRegressor": HuberRegressor(),
    "                     CatBoostRegressor": CatBoostRegressor(verbose=0),
    "             OrthogonalMatchingPursuit": OrthogonalMatchingPursuit()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.
                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                         BayesianRidge trained.
                        HuberRegressor trained.
                     CatBoostRegressor trained.
             OrthogonalMatchingPursuit trained.


In [9]:
#EVALUATE MODEL ON R SQUARED - HIGHER IS BETTER

for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))
    

                     Linear Regression R^2 Score: 0.46372
 Linear Regression (L2 Regularization) R^2 Score: 0.46209
 Linear Regression (L1 Regularization) R^2 Score: 0.27848
                   K-Nearest Neighbors R^2 Score: 0.80900
                        Neural Network R^2 Score: -6.21105
Support Vector Machine (Linear Kernel) R^2 Score: -292.74801
   Support Vector Machine (RBF Kernel) R^2 Score: 0.28180
                         Decision Tree R^2 Score: 0.89764
                         Random Forest R^2 Score: 0.97741
                     Gradient Boosting R^2 Score: 0.97603
                               XGBoost R^2 Score: 0.97932
                         BayesianRidge R^2 Score: 0.46174
                        HuberRegressor R^2 Score: -2.11969
                     CatBoostRegressor R^2 Score: 0.97932
             OrthogonalMatchingPursuit R^2 Score: 0.39464


In [11]:
#EVALUATE MODEL ON RMSE - LOWER IS BETTER

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(name + " Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
    y_train_pred = model.predict(X_train)
    print(name + " Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))
    
    errors = abs(y_pred - y_test)

    # Display the performance metrics
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'Rand.')

    mape = np.mean(100 * (errors / y_test))
    accuracy = 100 - mape

    print('Accuracy:', round(accuracy, 2), '%.')

                     Linear Regression Test RMSE: 1.53024
                     Linear Regression Train RMSE: 1.41372
Mean Absolute Error: 1.15 Rand.
Accuracy: 74.31 %.
 Linear Regression (L2 Regularization) Test RMSE: 1.53257
 Linear Regression (L2 Regularization) Train RMSE: 1.41424
Mean Absolute Error: 1.15 Rand.
Accuracy: 74.07 %.
 Linear Regression (L1 Regularization) Test RMSE: 1.77496
 Linear Regression (L1 Regularization) Train RMSE: 1.70080
Mean Absolute Error: 1.24 Rand.
Accuracy: 69.92 %.
                   K-Nearest Neighbors Test RMSE: 0.91322
                   K-Nearest Neighbors Train RMSE: 0.77752
Mean Absolute Error: 0.66 Rand.
Accuracy: 86.95 %.
                        Neural Network Test RMSE: 5.61129
                        Neural Network Train RMSE: 5.22232
Mean Absolute Error: 3.99 Rand.
Accuracy: 25.52 %.
Support Vector Machine (Linear Kernel) Test RMSE: 35.81388
Support Vector Machine (Linear Kernel) Train RMSE: 27.00464
Mean Absolute Error: 12.13 Rand.
Accuracy

In [ ]:
# GENERATE CSV FOR SUBMISSION TO KAGGLE
# DON'T SUBMIT ALL, ONLY THOSE WHICH DID GOOD ON EVALUATION TO CONFIRM

df = preprocess_inputs(test)

Xs = list(df.columns)
Xs.remove('Index')

X_test = df[Xs]

for name, model in models.items():
    y_pred = model.predict(X_test)
    d = pd.DataFrame(y_pred, columns =['avg_price_per_kg'])
    dff = pd.concat([df['Index'], d], axis=1)
    dff = dff.set_index('Index')
    
    match= re.findall('[A-Z]', name)    
    matchno = re.findall('[0-9]+', name)   
    file_name = ''.join(match) + ''.join(matchno)
    if file_name == 'XGB':
        dff.to_csv(file_name + '.csv')

# FEATURE SELECTION

In [13]:
# FEAUTURE IMPORTANCES

df = train.drop('avg_price_per_kg', axis = 1)
features = preprocess_inputs(df)
feature_list = list(features.columns)


for name, model in models.items():
    if name == "                               XGBoost":
        # Get numerical feature importances
        importances = list(model.feature_importances_)

        # List of tuples with variable and importance
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

        # Sort the feature importances by most important first
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

        # Print out the feature and importances 
        [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];



Variable: Weight_Kg            Importance: 0.3799999952316284
Variable: Low_Price            Importance: 0.17000000178813934
Variable: High_Price           Importance: 0.15000000596046448
Variable: Province_NATAL       Importance: 0.11999999731779099
Variable: year                 Importance: 0.029999999329447746
Variable: Sales_Total          Importance: 0.019999999552965164
Variable: Container_JE090      Importance: 0.019999999552965164
Variable: Container_JG110      Importance: 0.019999999552965164
Variable: Size_Grade           Importance: 0.009999999776482582
Variable: Total_Kg_Sold        Importance: 0.009999999776482582
Variable: Stock_On_Hand        Importance: 0.009999999776482582
Variable: month                Importance: 0.009999999776482582
Variable: Province_EASTERN_CAPE Importance: 0.009999999776482582
Variable: Province_W_CAPE_BERGRIVER_ETC Importance: 0.009999999776482582
Variable: Container_EC120      Importance: 0.009999999776482582
Variable: Container_M4183      Impo

In [14]:
#SELECTING IMPORTANT FEATURES

df = preprocess_inputs(train, return_df=True)


y = df['avg_price_per_kg']
X = df.drop('avg_price_per_kg', axis=1)

# GET IMPORTANT COLUMN NAMES

important = []
for i in ['Low_Price', 'Total_Kg_Sold', 'Container_IA400', 'Weight_Kg', 'Total_Qty_Sold', 'High_Price', 'Sales_Total']:
    A = [col for col in df.columns if i in col]
    important.append(A)
    
important_list = [item for sublist in important for item in sublist]

# IMPORTANT DATAFRAME
X_imp = X[important_list]


In [15]:
# TEST NEW MODEL WITH IMPORTANT FEATURES ONLY

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.02, shuffle=False, random_state=seed)

xgb = XGBRegressor(max_depth=3,min_child_weight=3,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=442, learning_rate=0.1, random_state= seed)

xgb.fit(X_imp_train, y_train)

print("Trained.")

Trained.


In [16]:
# CHECK PERFORMANCE METRICS

pred = xgb.predict(X_imp_test)

errors = abs(pred - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

mape = np.mean(100 * (errors / y_test))
accuracy = 100 - mape

print('Accuracy:', round(accuracy, 2), '%.')
print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,pred))))
y_train_pred = xgb.predict(X_imp_train)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))

Mean Absolute Error: 0.22 degrees.
Accuracy: 95.75 %.
Test RMSE: 0.31163
Train RMSE: 0.25125


Lets compare all the models with their previous score rather.

In [17]:
# TRAIN MODELS AGAIN BUT WITH THE SUBSET CREATER ABOVE

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.02, shuffle=False, random_state=seed)

models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(random_state= seed),
    "                         Random Forest": RandomForestRegressor(n_estimators=442, random_state= seed),
    "                     Gradient Boosting": GradientBoostingRegressor(n_estimators=442, random_state= seed),
    "                               XGBoost": XGBRegressor(max_depth=3,min_child_weight=3,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=442, learning_rate=0.1, random_state= seed),
    "                         BayesianRidge": BayesianRidge(),
    "                        HuberRegressor": HuberRegressor(),
    "                     CatBoostRegressor": CatBoostRegressor(verbose=0),
    "             OrthogonalMatchingPursuit": OrthogonalMatchingPursuit()
}

for name, model in models.items():
    model.fit(X_imp_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.
                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                         BayesianRidge trained.
                        HuberRegressor trained.
                     CatBoostRegressor trained.
             OrthogonalMatchingPursuit trained.


In [18]:
#EVALUATE SQUARED - HIGHER IS BETTER

for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_imp_test, y_test)))
 

                     Linear Regression R^2 Score: 0.38435
 Linear Regression (L2 Regularization) R^2 Score: 0.20706
 Linear Regression (L1 Regularization) R^2 Score: 0.16335
                   K-Nearest Neighbors R^2 Score: 0.92132
                        Neural Network R^2 Score: -31.49171
Support Vector Machine (Linear Kernel) R^2 Score: -1066.10783
   Support Vector Machine (RBF Kernel) R^2 Score: 0.28546
                         Decision Tree R^2 Score: 0.96323
                         Random Forest R^2 Score: 0.97814
                     Gradient Boosting R^2 Score: 0.98218
                               XGBoost R^2 Score: 0.97776
                         BayesianRidge R^2 Score: 0.38137
                        HuberRegressor R^2 Score: -30.83005
                     CatBoostRegressor R^2 Score: 0.98433
             OrthogonalMatchingPursuit R^2 Score: 0.02380


In [19]:
#EVALUATE MODEL ON RMSE

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.02, shuffle=False, random_state=seed)

for name, model in models.items():
    y_pred = model.predict(X_imp_test)
    print(name + " Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
    y_train_pred = model.predict(X_imp_train)
    print(name + " Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error( y_train,y_train_pred))))
    
    errors = abs(y_pred - y_test)

    # Display the performance metrics
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

    mape = np.mean(100 * (errors / y_test))
    accuracy = 100 - mape

    print('Accuracy:', round(accuracy, 2), '%.')

                     Linear Regression Test RMSE: 1.63958
                     Linear Regression Train RMSE: 1.65937
Mean Absolute Error: 1.19 degrees.
Accuracy: 72.37 %.
 Linear Regression (L2 Regularization) Test RMSE: 1.86073
 Linear Regression (L2 Regularization) Train RMSE: 1.75260
Mean Absolute Error: 1.3 degrees.
Accuracy: 68.58 %.
 Linear Regression (L1 Regularization) Test RMSE: 1.91134
 Linear Regression (L1 Regularization) Train RMSE: 1.78442
Mean Absolute Error: 1.34 degrees.
Accuracy: 67.65 %.
                   K-Nearest Neighbors Test RMSE: 0.58613
                   K-Nearest Neighbors Train RMSE: 0.62157
Mean Absolute Error: 0.41 degrees.
Accuracy: 92.0 %.
                        Neural Network Test RMSE: 11.91105
                        Neural Network Train RMSE: 9.84465
Mean Absolute Error: 6.91 degrees.
Accuracy: -8.75 %.
Support Vector Machine (Linear Kernel) Test RMSE: 68.26028
Support Vector Machine (Linear Kernel) Train RMSE: 52.31970
Mean Absolute Error: 28.93 

It apears the majority of the best performing models from our initial training have improved. Lets improve them a bit more.

# Ensemble Stacking

In [43]:
# NOW ENSEMBLE STACKING

# CHOOSE BEST MODELS FROM EARLIER SCORES


rf = RandomForestRegressor(n_estimators= 442, random_state= seed)
gb =GradientBoostingRegressor(random_state= seed, n_estimators=442)
xgb = XGBRegressor(max_depth=3,min_child_weight=3,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=442, learning_rate=0.1, random_state= seed)
cb= CatBoostRegressor(verbose=0, learning_rate=0.1)
dt = DecisionTreeRegressor(random_state= seed)

meta_learner_reg = CatBoostRegressor(verbose=0, learning_rate=0.1)


model_4st = [("rf", rf),("gb", gb),("xgb", xgb), ('cb', cb), ('dt', dt)]

s_reg = StackingRegressor(estimators=model_4st, final_estimator= meta_learner_reg, passthrough = True, cv= 4)


In [21]:
# TRAIN ON DATAFRAME WITH 7 COLUMNS(MOST IMPORTANT)

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.02, shuffle=False, random_state=seed)

s_reg.fit(X_imp_train,y_train)
print("Stacked model fitted.")

Stacked model fitted.


In [218]:
X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.02, shuffle=False, random_state=seed)

In [22]:
# METRICS OF STACKING REGRESSOR

X_train, X_test, y_train, y_test = preprocess_inputs(train)

y_pred = s_reg.predict(X_imp_test)
rsq = s_reg.score(X_imp_test, y_test)
print("R^2 Score: ", rsq)

print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error( y_test,y_pred))))
y_train_pred = s_reg.predict(X_imp_train)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))
 
errors = abs(y_pred - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / y_test))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

R^2 Score:  0.980916920053333
Test RMSE: 0.28866
Train RMSE: 0.23686
Mean Absolute Error: 0.2 degrees.
Accuracy: 96.32 %.


In [225]:
df = preprocess_inputs(test)
Xs = list(df.columns)
Xs.remove('Index')

X_test = df[Xs]

x_t = X_test[important_list]

y_pred = s_reg.predict(x_t)
d = pd.DataFrame(y_pred, columns =['avg_price_per_kg'])
dff = pd.concat([df['Index'], d], axis=1)
dff = dff.set_index('Index')
dff.to_csv('StackeD27.csv')

In [23]:
important = []
for i in ['Low_Price', 'Total_Kg_Sold', 'Container_IA400', 'Weight_Kg', 'Total_Qty_Sold', 'High_Price', 'Sales_Total']:
    A = [col for col in df.columns if i in col]
    important.append(A)
    
important_list = [item for sublist in important for item in sublist]

# IMPORTANT DATAFRAME
X_imp = X[important_list]

In [44]:
# USING KFOLD

df = preprocess_inputs(train, return_df=True)

important = []
for i in ['Low_Price', 'Total_Kg_Sold', 'Container_IA400', 'Weight_Kg', 'Total_Qty_Sold', 'High_Price', 'Sales_Total']:
    A = [col for col in df.columns if i in col]
    important.append(A)
    
important_list = [item for sublist in important for item in sublist]


def split_data_kf(df,K):
    
    y = df['avg_price_per_kg']
    X = df[important_list]  # SUBSET FOR IMPORTANT ROWS
    
    kf = KFold(n_splits=K, shuffle = False)
    indices = []
    
    for train_index, test_index in kf.split(X):
        indices.append((train_index, test_index))
        
    return indices


def get_best_kfmodel(df,data_indices):
    y = df['avg_price_per_kg']
    X = df[important_list]
    
    RMSE = []
    trainRSME =[]
    
    for (train_indices,test_indices) in data_indices:
        X_train, y_train = X.iloc[train_indices,:],y.iloc[train_indices]
        X_test, y_test = X.iloc[test_indices,:], y.iloc[test_indices]
        
        model = StackingRegressor(estimators=model_4st, final_estimator= meta_learner_reg, passthrough = True, cv= 4)
        
        model.fit(X_train, y_train) 

        y_pred = model.predict(X_test)     
        
        rmse = np.sqrt(metrics.mean_squared_error(y_test ,y_pred))
        RMSE.append(rmse)
        
        y_train_pred = model.predict(X_train)
        
        train_rsme = np.sqrt(metrics.mean_squared_error( y_train,y_train_pred))
        trainRSME.append(train_rsme)
        
    best = RMSE.index(min(RMSE))
    best_indicies = data_indices[best]
    
    
    X_train, y_train = X.iloc[best_indicies[0],:],y.iloc[best_indicies[0]]
    X_test, y_test = X.iloc[best_indicies[1],:], y.iloc[best_indicies[1]]
    
    model = StackingRegressor(estimators=model_4st, final_estimator= meta_learner_reg, passthrough = True, cv= 4)
    
    model.fit(X_train, y_train)       
          
    return trainRSME, RMSE
       

In [45]:
df = preprocess_inputs(train, return_df = True)
data_indices = split_data_kf(df,4)

get_best_kfmodel(df,data_indices)

([0.159509151550616,
  0.29588091344308826,
  0.24568148722113006,
  0.26808761513218066],
 [0.6827825330873297,
  0.3950390501902279,
  0.4077639508599942,
  0.4600179831132657])

In [563]:
#important_list = [item for sublist in important for item in sublist]

#train_df = preprocess_inputs(train, return_df = True)
#data_indices = split_data_kf(train_df,4)
#model = get_best_kfmodel(train_df,data_indices)

df = preprocess_inputs(test)
Xs = list(df.columns)
Xs.remove('Index')

X_test = df[Xs]
X_Test_imp = X_test[important_list]

y_pred = xgb.predict(X_test)
d = pd.DataFrame(y_pred, columns =['avg_price_per_kg'])
dff = pd.concat([df['Index'], d], axis=1)
dff = dff.set_index('Index')
dff.to_csv('xgb.csv')

## Hyperparameter Tuning
### XGBoostRegressor

In [246]:

import xgboost as xgb

X_train, X_test, y_train, y_test = preprocess_inputs(train)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

params['eval_metric'] = "rmse"
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10)


print("Best RMSE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))


[0]	Test-rmse:4.24003
[1]	Test-rmse:2.95083
[2]	Test-rmse:2.07685
[3]	Test-rmse:1.45692
[4]	Test-rmse:1.03832
[5]	Test-rmse:0.74636
[6]	Test-rmse:0.55419
[7]	Test-rmse:0.44213
[8]	Test-rmse:0.37587
[9]	Test-rmse:0.34743
[10]	Test-rmse:0.33719
[11]	Test-rmse:0.32814
[12]	Test-rmse:0.31795
[13]	Test-rmse:0.32059
[14]	Test-rmse:0.32324
[15]	Test-rmse:0.33245
[16]	Test-rmse:0.32408
[17]	Test-rmse:0.31866
[18]	Test-rmse:0.32190
[19]	Test-rmse:0.31826
[20]	Test-rmse:0.31532
[21]	Test-rmse:0.31433
[22]	Test-rmse:0.31612
[23]	Test-rmse:0.31563
[24]	Test-rmse:0.31012
[25]	Test-rmse:0.31401
[26]	Test-rmse:0.31333
[27]	Test-rmse:0.31418
[28]	Test-rmse:0.31419
[29]	Test-rmse:0.31111
[30]	Test-rmse:0.31438
[31]	Test-rmse:0.31141
[32]	Test-rmse:0.31090
[33]	Test-rmse:0.31058
[34]	Test-rmse:0.31162
Best RMSE: 0.31 with 25 rounds


In [250]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=1,
    nfold=4,
    metrics={'rmse'},
    early_stopping_rounds=10
)
cv_results

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,4.759979,0.010120,4.759272,0.042933
1,3.434726,0.012340,3.438186,0.044566
2,2.527619,0.013353,2.537529,0.049325
3,1.917172,0.017705,1.934875,0.060541
4,1.499747,0.021379,1.519129,0.069289
...,...,...,...,...
241,0.157172,0.002792,0.531166,0.084704
242,0.156778,0.002842,0.531033,0.084783
243,0.156232,0.003002,0.530817,0.084777
244,0.155791,0.003105,0.530773,0.084872


In [251]:
cv_results['test-rmse-mean'].min()

0.5301345

In [252]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(3,4)
    for min_child_weight in range(3,7)
]

# Define initial best params and RMSE
min_rmse = float("Inf")
best_params = None

for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=1,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10)
    
    # Update best RMSE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth,min_child_weight)
        
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

CV with max_depth=3, min_child_weight=3
	RMSE 0.5082641999999999 for 544 rounds
CV with max_depth=3, min_child_weight=4
	RMSE 0.5196882 for 505 rounds
CV with max_depth=3, min_child_weight=5
	RMSE 0.5191416 for 431 rounds
CV with max_depth=3, min_child_weight=6
	RMSE 0.5340222000000001 for 253 rounds
Best params: 3, 3, RMSE: 0.5082641999999999


In [255]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [10/10. for i in range(10,11)]
    for colsample in [10/10. for i in range(10,11)]
]

min_rmse = float("Inf")
best_params = None


for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(subsample, colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=1,
        nfold=4,
        metrics={'rmse'},
        early_stopping_rounds=10)
    
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample,colsample)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))


CV with subsample=1.0, colsample=1.0
	RMSE 0.5007085 for 761 rounds
Best params: 1.0, 1.0, RMSE: 0.5007085


In [259]:
%time

min_rmse = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    
    %time cv_results = xgb.cv(params, dtrain,num_boost_round=num_boost_round,seed=1, nfold=4, metrics=['rmse'], early_stopping_rounds=10)
    
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds\n".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = eta
print("Best params: {}, RMSE: {}".format(best_params, min_rmse))

Wall time: 0 ns
CV with eta=0.3
Wall time: 3.62 s
	RMSE 0.5007085 for 761 rounds

CV with eta=0.2
Wall time: 2.84 s
	RMSE 0.5086425 for 618 rounds

CV with eta=0.1
Wall time: 4.56 s
	RMSE 0.51088125 for 996 rounds

CV with eta=0.05
Wall time: 4.7 s
	RMSE 0.5370935 for 998 rounds

CV with eta=0.01
Wall time: 4.57 s
	RMSE 0.63481625 for 998 rounds

CV with eta=0.005
Wall time: 4.64 s
	RMSE 0.72191725 for 998 rounds

Best params: 0.3, RMSE: 0.5007085


In [260]:
params['eta'] = .1

params

{'max_depth': 3,
 'min_child_weight': 6,
 'eta': 0.1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse'}

In [261]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best RMSE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[0]	Test-rmse:5.48784
[1]	Test-rmse:4.93565
[2]	Test-rmse:4.43379
[3]	Test-rmse:3.98805
[4]	Test-rmse:3.59204
[5]	Test-rmse:3.22469
[6]	Test-rmse:2.91537
[7]	Test-rmse:2.63697
[8]	Test-rmse:2.38234
[9]	Test-rmse:2.15274
[10]	Test-rmse:1.95026
[11]	Test-rmse:1.76234
[12]	Test-rmse:1.58760
[13]	Test-rmse:1.45094
[14]	Test-rmse:1.32922
[15]	Test-rmse:1.20769
[16]	Test-rmse:1.11868
[17]	Test-rmse:1.02924
[18]	Test-rmse:0.95382
[19]	Test-rmse:0.88946
[20]	Test-rmse:0.83332
[21]	Test-rmse:0.78195
[22]	Test-rmse:0.74035
[23]	Test-rmse:0.70253
[24]	Test-rmse:0.66709
[25]	Test-rmse:0.63953
[26]	Test-rmse:0.61081
[27]	Test-rmse:0.59781
[28]	Test-rmse:0.58291
[29]	Test-rmse:0.56744
[30]	Test-rmse:0.55616
[31]	Test-rmse:0.54495
[32]	Test-rmse:0.53250
[33]	Test-rmse:0.52379
[34]	Test-rmse:0.51979
[35]	Test-rmse:0.50646
[36]	Test-rmse:0.49441
[37]	Test-rmse:0.48813
[38]	Test-rmse:0.48440
[39]	Test-rmse:0.47901
[40]	Test-rmse:0.47588
[41]	Test-rmse:0.46967
[42]	Test-rmse:0.46096
[43]	Test-rmse:0.4610

In [262]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-rmse:5.48784
[1]	Test-rmse:4.93565
[2]	Test-rmse:4.43379
[3]	Test-rmse:3.98805
[4]	Test-rmse:3.59204
[5]	Test-rmse:3.22469
[6]	Test-rmse:2.91537
[7]	Test-rmse:2.63697
[8]	Test-rmse:2.38234
[9]	Test-rmse:2.15274
[10]	Test-rmse:1.95026
[11]	Test-rmse:1.76234
[12]	Test-rmse:1.58760
[13]	Test-rmse:1.45094
[14]	Test-rmse:1.32922
[15]	Test-rmse:1.20769
[16]	Test-rmse:1.11868
[17]	Test-rmse:1.02924
[18]	Test-rmse:0.95382
[19]	Test-rmse:0.88946
[20]	Test-rmse:0.83332
[21]	Test-rmse:0.78195
[22]	Test-rmse:0.74035
[23]	Test-rmse:0.70253
[24]	Test-rmse:0.66709
[25]	Test-rmse:0.63953
[26]	Test-rmse:0.61081
[27]	Test-rmse:0.59781
[28]	Test-rmse:0.58291
[29]	Test-rmse:0.56744
[30]	Test-rmse:0.55616
[31]	Test-rmse:0.54495
[32]	Test-rmse:0.53250
[33]	Test-rmse:0.52379
[34]	Test-rmse:0.51979
[35]	Test-rmse:0.50646
[36]	Test-rmse:0.49441
[37]	Test-rmse:0.48813
[38]	Test-rmse:0.48440
[39]	Test-rmse:0.47901
[40]	Test-rmse:0.47588
[41]	Test-rmse:0.46967
[42]	Test-rmse:0.46096
[43]	Test-rmse:0.4610

In [263]:
y_pred = best_model.predict(dtest)
print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
y_train_pred = best_model.predict(dtrain)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))

Test RMSE: 0.31247
Train RMSE: 0.39757


In [266]:
params


{'max_depth': 3,
 'min_child_weight': 6,
 'eta': 0.1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse'}

In [277]:
X_train, X_test, y_train, y_test = preprocess_inputs(train)

max_depth = 3
min_child_weight = 3
subsample = 1
colsample_bytree = 1
objective = 'reg:squarederror'
num_estimators = 442
learning_rate = 0.1

xgb = XGBRegressor(max_depth=max_depth,
                min_child_weight=min_child_weight,
                subsample=subsample,
                colsample_bytree=colsample_bytree,
                objective=objective,
                n_estimators=num_estimators,
                learning_rate=learning_rate, random_state= seed)

xgb.fit(X_train, y_train)
print('trained.')

trained.


In [278]:
y_pred = xgb.predict(X_test)
print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
y_train_pred = xgb.predict(X_train)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))

Test RMSE: 0.30049
Train RMSE: 0.24729


In [ ]:
(max_depth=3,min_child_weight=3,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=442, learning_rate=0.1, random_state= seed)